In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from zipfile import ZipFile, ZIP_DEFLATED
import os
import datetime
from datetime import date, datetime

# get current directory to use absolute file paths
current_directory = os.getcwd() 

# path to zip
zip_path = os.path.join(current_directory,"Resources","US_Accidents.zip")

# file we care about in the zip
file_name = "US_Accidents_Cleaned.csv"

In [2]:
# access zip file
with ZipFile(zip_path) as zin:
    # read csv
    accidents_df = pd.read_csv(zin.open(file_name))
    
accidents_df.head()

,ID,Severity,Start_Time,Start_Lat,Start_Lng,City,County,State,Zipcode,Sunrise_Sunset
0,A-1,3,2016-02-08 05:46:00,39.865147,-84.058723,Dayton,Montgomery,OH,45424,Night
1,A-2,2,2016-02-08 06:07:59,39.928059,-82.831184,Reynoldsburg,Franklin,OH,43068-3402,Night
2,A-3,2,2016-02-08 06:49:27,39.063148,-84.032608,Williamsburg,Clermont,OH,45176,Night
3,A-4,3,2016-02-08 07:23:34,39.747753,-84.205582,Dayton,Montgomery,OH,45417,Night
4,A-5,2,2016-02-08 07:39:07,39.627781,-84.188354,Dayton,Montgomery,OH,45459,Day


In [3]:
# Colleen's code starts here

In [4]:
# Colleen's code ends here

In [5]:
# Ken's code starts here
#Grouping Data Into Seasons
# In order to group dates into seasons, we need to define seasons then group the dates into seasons column using  Meteorological seasons classifications for northern hemisphere
#Spring runs from March 1 to May 31;
#Summer runs from June 1 to Aug.31;
#Fall (Autumn) runs from September 1st thru Nov.30th and
#Winter runs from December 1 to 28 Feb


month_seasons = {1: 'winter',
                 2: 'winter',
                 3: 'spring',
                 4: 'spring',
                 5: 'spring',
                 6: 'summer',
                 7: 'summer',
                 8: 'summer',
                 9: 'fall',
                 10: 'fall',
                 11: 'fall',
                 12: 'winter'}

accidents_df['seasons'] = pd.to_datetime(accidents_df['Start_Time']).dt.month.map(month_seasons)
accidents_df.tail(50000)

,ID,Severity,Start_Time,Start_Lat,Start_Lng,City,County,State,Zipcode,Sunrise_Sunset,seasons
3016399,A-3463781,2,2019-05-24 17:42:00,41.026940,-111.945715,Kaysville,Davis,UT,84037,Day,spring
3016400,A-3463782,2,2019-05-24 17:51:00,38.756321,-111.382694,Richfield,Sevier,UT,84701,Day,spring
3016401,A-3463783,2,2019-05-24 18:10:00,40.122100,-111.659290,Spanish Fork,Utah,UT,84660,Day,spring
3016402,A-3463784,3,2019-05-24 19:53:13,39.823100,-104.967300,Denver,Adams,CO,80229,Day,spring
3016403,A-3463785,2,2019-05-24 20:29:00,40.749770,-111.605100,Park City,Summit,UT,84098,Day,spring
...,...,...,...,...,...,...,...,...,...,...,...
3066394,A-3513776,2,2019-08-23 18:03:25,34.002480,-117.379360,Riverside,Riverside,CA,92501,Day,summer
3066395,A-3513777,2,2019-08-23 19:11:30,32.766960,-117.148060,San Diego,San Diego,CA,92108,Day,summer
3066396,A-3513778,2,2019-08-23 19:00:21,33.775450,-117.847790,Orange,Orange,CA,92866,Day,summer
3066397,A-3513779,2,2019-08-23 19:00:21,33.992460,-118.403020,Culver City,Los Angeles,CA,90230,Day,summer


In [ ]:
# Ken Code Ends

In [6]:
# Eric's code starts here

In [7]:
# Chris's code ends here

In [8]:
# Eric's code ends here

In [9]:
# Chris's code starts here